In [1]:
import pymysql as sql

In [2]:
connection = sql.connect(
    host="localhost",
    user="root",
    password="343450",
    database="Nutrition_Paradox"
)

cursor = connection.cursor()

In [3]:
def create_tables():
    cursor.execute(""" CREATE TABLE IF NOT EXISTS obesity(
                   region TEXT,
                   gender TEXT,
                   year INTEGER,
                   low REAL,
                   high REAL,
                   mean_estimate REAL,
                   country TEXT,
                   age_group TEXT,
                   ci_width REAL,
                   obesity_level TEXT);
                   """)
    
    cursor.execute(""" CREATE TABLE IF NOT EXISTS malnutrition(
                   region TEXT,
                   gender TEXT,
                   year INTEGER,
                   low REAL,
                   high REAL,
                   mean_estimate REAL,
                   country TEXT,
                   age_group TEXT,
                   ci_width REAL,
                   malnutrition_level TEXT);
                   """)
    connection.commit()

create_tables()

In [4]:
import pandas as pd

df_obesity = pd.read_csv("df_obesity_cleaned.csv")
df_malnutrition = pd.read_csv("df_malnutrition_cleaned.csv")

In [5]:
for _, row in df_obesity.iterrows():
    cursor.execute(""" INSERT INTO obesity(region,gender,year,low,high,mean_estimate,country,age_group,ci_width,obesity_level) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)""",(
        row['Region'],row['Gender'],row['Year'],row['Low'],row['High'],row['Mean_Estimate'],row['Country'],row['age_group'],row['CI_Width'],row['obesity_level']))
    
for _, row in df_malnutrition.iterrows():
    cursor.execute(""" INSERT INTO malnutrition(region,gender,year,low,high,mean_estimate,country,age_group,ci_width,malnutrition_level) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)""",(
        row['Region'],row['Gender'],row['Year'],row['Low'],row['High'],row['Mean_Estimate'],row['Country'],row['age_group'],row['CI_Width'],row['malnutrition_level']))
    
connection.commit()

In [6]:
#Top 5 regions with the highest average obesity levels in the most recent year(2022)
query = """SELECT region, AVG(obesity_level) AS avg_obesity FROM obesity WHERE year = 2022 GROUP BY region ORDER BY avg_obesity DESC LIMIT 5;"""
cursor.execute(query)
results = cursor.fetchall()

for row in results:
    print(f"Region: {row[0]}, Avg Obesity: {row[1]:.2f}")

Region: Western Pacific, Avg Obesity: 0.00
Region: Europe, Avg Obesity: 0.00
Region: Africa, Avg Obesity: 0.00
Region: South-East Asia Region, Avg Obesity: 0.00
Region: Americas, Avg Obesity: 0.00


In [7]:
#Top 5 countries with highest obesity estimates
query = """SELECT country, AVG(mean_estimate) AS avg_obesity FROM obesity GROUP BY country ORDER BY avg_obesity DESC LIMIT 5;"""
cursor.execute(query)
results = cursor.fetchall()

for row in results:
    print(f"Country: {row[0]}, Avg Obesity: {row[1]:.2f}")

Country: Niue, Avg Obesity: 33.07
Country: Cook Islands, Avg Obesity: 32.25
Country: Tokelau, Avg Obesity: 30.32
Country: Nauru, Avg Obesity: 30.30
Country: Tonga, Avg Obesity: 29.28


In [8]:
#Obesity trend in India over the years(Mean_estimate)
query = """SELECT year, AVG(mean_estimate) AS avg_obesity FROM obesity WHERE country = 'India' GROUP BY year ORDER BY year;"""
cursor.execute(query)
results = cursor.fetchall()

results

((2012, 2.4731781416666667),
 (2013, 2.6346529999999997),
 (2014, 2.7987494500000003),
 (2015, 2.972813325),
 (2016, 3.1595759500000002),
 (2017, 3.362174975),
 (2018, 3.5792918250000003),
 (2019, 3.8090129416666656),
 (2020, 4.0511491416666665),
 (2021, 4.307815533333334),
 (2022, 4.5791794249999995))

In [9]:
#Average obesity by gender
query ="""SELECT gender, AVG(mean_estimate) AS avg_obesity FROM obesity GROUP BY gender;"""
cursor.execute(query)
results = cursor.fetchall()
results

(('Both', 12.417273217540586),
 ('Female', 12.116017301377912),
 ('Male', 12.507732618309204))

In [10]:
#Country count by obesity level category and age group
query ="""SELECT age_group, obesity_level, COUNT(DISTINCT country) AS country_count FROM obesity GROUP BY age_group, obesity_level ORDER BY age_group, obesity_level;"""
cursor.execute(query)
results = cursor.fetchall()
results


(('adults', 'High', 94),
 ('adults', 'Low', 176),
 ('adults', 'Moderate', 99),
 ('children', 'High', 11),
 ('children', 'Low', 210),
 ('children', 'Moderate', 26))

In [11]:
#Top 5 least reliable countries (highest average CI_Width)
query = """SELECT country, AVG(ci_width) AS avg_ci_width FROM obesity GROUP BY country ORDER BY avg_ci_width DESC LIMIT 5;"""
cursor.execute(query)
results = cursor.fetchall()
results

(('Bermuda', 20.549668542471608),
 ('Antigua and Barbuda', 19.220431725331448),
 ('American Samoa', 19.013790021780316),
 ('Puerto Rico', 18.44155894337122),
 ('Syrian Arab Republic', 18.20622462528409))

In [12]:
#Top 5 most consistent countries (lowest average CI_Width)
query = """SELECT country, AVG(ci_width) AS avg_ci_width FROM obesity GROUP BY country ORDER BY avg_ci_width ASC LIMIT 5;"""
cursor.execute(query)
results = cursor.fetchall()
results

(('GLOBAL', 0.7089528560606064),
 ('WB_LMI', 0.861059624242424),
 ('SEAR', 1.0177745681818178),
 ('India', 1.1401325691666664),
 ('Japan', 1.1559030522727274))

In [13]:
#Average obesity by age group
query = """SELECT age_group, AVG(obesity_level) AS avg_obesity FROM obesity GROUP BY age_group ORDER BY avg_obesity DESC;"""
cursor.execute(query)
results = cursor.fetchall()
results

(('adults', 0.0), ('children', 0.0))

In [14]:
#Top 10 Countries with consistent low obesity (low average + low CI)over the years
query = """SELECT country,AVG(mean_estimate) AS avg_obesity,AVG(ci_width) AS avg_ci_width FROM obesity GROUP BY country ORDER BY avg_obesity ASC, avg_ci_width ASC LIMIT 10;"""
cursor.execute(query)
results = cursor.fetchall()
results

(('Timor-Leste', 0.994067426287879, 1.2735691667424243),
 ('Ethiopia', 1.4919943762878793, 1.5894139543939394),
 ('Burkina Faso', 1.721495976818182, 1.5315138911363642),
 ('Rwanda', 1.7671937959090915, 2.3605786345530304),
 ('Bangladesh', 1.9766725870454547, 1.6635064317424242),
 ('Madagascar', 1.9841098104545452, 4.3568854983106045),
 ('Burundi', 2.304665463636365, 5.1980434951515155),
 ('Malawi', 2.391323871439394, 2.6327135187878787),
 ('Nepal', 2.49425472628788, 3.0980508735606063),
 ('Eritrea', 2.7705596740909093, 5.870395333409088))

In [15]:
#Countries where female obesity exceeds male by large margin (same       year)
query = """SELECT f.country,f.year,f.mean_estimate AS female_obesity,m.mean_estimate AS male_obesity,(f.mean_estimate - m.mean_estimate) AS diff FROM obesity f
JOIN obesity m ON f.country = m.country AND f.year = m.year AND f.age_group = m.age_group WHERE f.gender = 'Female' AND m.gender = 'Male' AND (f.mean_estimate - m.mean_estimate) > 5 -- threshold for "large margin" ORDER BY diff DESC;"""
cursor.execute(query)
results = cursor.fetchall()
results

(('Iran, Islamic Republic of', 2020, 30.759203, 17.487641, 13.271562),
 ('Iraq', 2020, 36.57509561249999, 28.33553, 8.239565612499995),
 ('Jordan', 2017, 36.57509561249999, 29.548621, 7.026474612499992),
 ('Botswana', 2020, 25.232658, 7.2276138, 18.0050442),
 ('Senegal', 2012, 10.20498, 2.0968506, 8.108129400000001),
 ('Namibia', 2018, 19.443649, 7.406456, 12.037193),
 ('Paraguay', 2016, 29.250786, 23.505804, 5.744982),
 ('Liberia', 2014, 13.738862, 7.1819074, 6.556954599999999),
 ('Lesotho', 2018, 27.536297, 5.6822147, 21.8540823),
 ('Bolivia, Plurinational State of',
  2012,
  23.75396,
  14.607212,
  9.146747999999999),
 ('Bhutan', 2020, 13.76404, 7.9906911, 5.773348899999999),
 ('Brazil', 2013, 23.160377, 16.42217, 6.738206999999999),
 ('Barbados', 2014, 36.57509561249999, 22.186777, 14.388318612499994),
 ('Ukraine', 2018, 31.81406, 20.458104, 11.355956000000003),
 ('Saint Lucia', 2017, 36.57509561249999, 17.167157, 19.407938612499994),
 ('Nigeria', 2017, 12.150143, 5.9263818, 6.22

In [16]:
#Global average obesity percentage per year
query = """SELECT year,AVG(mean_estimate) AS global_avg_obesity FROM obesity GROUP BY year ORDER BY year;"""
cursor.execute(query)
results = cursor.fetchall()
results

((2012, 10.744584998227161),
 (2013, 11.027700897848215),
 (2014, 11.323063791424586),
 (2015, 11.63003049358431),
 (2016, 11.949715261613084),
 (2017, 12.284496542818449),
 (2018, 12.632639987529755),
 (2019, 12.993617495350211),
 (2020, 13.365445191628016),
 (2021, 13.74300532985019),
 (2022, 14.122784846627988))

In [17]:
#Avg. malnutrition by age group
query = """SELECT age_group,AVG(mean_estimate) AS avg_malnutrition FROM malnutrition GROUP BY age_group ORDER BY avg_malnutrition DESC;"""
cursor.execute(query)
results = cursor.fetchall()
results

(('adults', 5.582712623767691), ('children', 4.983141235701341))

In [18]:
#Top 5 countries with highest malnutrition(mean_estimate)
query ="""SELECT country,AVG(mean_estimate) AS avg_malnutrition FROM malnutrition GROUP BY country ORDER BY avg_malnutrition DESC LIMIT 5;"""
cursor.execute(query)
results = cursor.fetchall()
results

(('India', 15.227341011363682),
 ('SEAR', 15.135994681818229),
 ('Sri Lanka', 15.06141804924247),
 ('Bangladesh', 14.861726895833367),
 ('WB_LMI', 14.389638990530326))

In [19]:
#Malnutrition trend in African region over the years
query ="""SELECT year, AVG(mean_estimate) AS avg_malnutrition FROM malnutrition WHERE region = 'Africa' GROUP BY year ORDER BY year;"""
cursor.execute(query)
results=cursor.fetchall()
results

((2012, 8.443636122222234),
 (2013, 8.357497705381942),
 (2014, 8.281869757552078),
 (2015, 8.216227478385425),
 (2016, 8.159171628906257),
 (2017, 8.107573971354153),
 (2018, 8.060153760069447),
 (2019, 8.01878449730903),
 (2020, 7.982142552777775),
 (2021, 7.949800003559032),
 (2022, 7.92160789244792))

In [20]:
#Gender-based average malnutrition
query ="""SELECT gender,AVG(mean_estimate) AS avg_malnutrition FROM malnutrition GROUP BY gender;"""
cursor.execute(query)
results = cursor.fetchall()
results

(('Male', 5.677133012842038),
 ('Female', 4.5480076417316155),
 ('Both', 5.173961593580118))

In [21]:
#Malnutrition level-wise (average CI_Width by age group)
query ="""SELECT malnutrition_level,age_group,AVG(ci_width) AS avg_ci_width FROM malnutrition GROUP BY malnutrition_level, age_group ORDER BY malnutrition_level, age_group;"""
cursor.execute(query)
results = cursor.fetchall()
results

(('High', 'adults', 8.501938770089282),
 ('High', 'children', 7.356356886788921),
 ('Low', 'adults', 2.631764294386123),
 ('Low', 'children', 4.385435799584495),
 ('Moderate', 'adults', 7.109511886735708),
 ('Moderate', 'children', 8.869173732345347))

In [22]:
#Yearly malnutrition change in specific countries(India, Nigeria, Brazil)
query = """SELECT country,year,AVG(mean_estimate) AS avg_malnutrition FROM malnutrition WHERE country IN ('India', 'Nigeria', 'Brazil') GROUP BY country, year ORDER BY country, year;"""
cursor.execute(query)
results=cursor.fetchall()
results

(('Brazil', 2012, 2.8386533000000003),
 ('Brazil', 2013, 2.7813881583333337),
 ('Brazil', 2014, 2.735034283333333),
 ('Brazil', 2015, 2.6989469583333334),
 ('Brazil', 2016, 2.6722411833333326),
 ('Brazil', 2017, 2.6535659499999995),
 ('Brazil', 2018, 2.641574491666667),
 ('Brazil', 2019, 2.6345942499999997),
 ('Brazil', 2020, 2.6294117166666666),
 ('Brazil', 2021, 2.626636775),
 ('Brazil', 2022, 2.626853608333333),
 ('India', 2012, 15.279193749999997),
 ('India', 2013, 15.279193749999997),
 ('India', 2014, 15.279193749999997),
 ('India', 2015, 15.279193749999997),
 ('India', 2016, 15.279193749999997),
 ('India', 2017, 15.279193749999997),
 ('India', 2018, 15.279193749999997),
 ('India', 2019, 15.279193749999997),
 ('India', 2020, 15.229901520833332),
 ('India', 2021, 15.109700624999997),
 ('India', 2022, 14.927598979166666),
 ('Nigeria', 2012, 10.856440741666667),
 ('Nigeria', 2013, 10.902669375),
 ('Nigeria', 2014, 10.971493658333335),
 ('Nigeria', 2015, 11.058089633333333),
 ('Nigeri

In [23]:
#Regions with lowest malnutrition averages
query = """SELECT region,AVG(mean_estimate) AS avg_malnutrition FROM malnutrition GROUP BY region ORDER BY avg_malnutrition ASC LIMIT 5;"""
cursor.execute(query)
results=cursor.fetchall()
results

(('Americas Region', 2.024236674242424),
 ('High Income', 2.322331581060607),
 ('Europe', 2.4290010327830105),
 ('Americas', 3.1737307278132665),
 ('Western Pacific', 3.205940339242424))

In [24]:
#Countries with increasing malnutrition (💡 Hint: Use MIN() and MAX()   on Mean_Estimate per country to compare early vs. recent malnutrition levels, and filter where the difference is positive using HAVING.)
query = """SELECT country,MIN(mean_estimate) AS min_malnutrition,MAX(mean_estimate) AS max_malnutrition,
    (MAX(mean_estimate) - MIN(mean_estimate)) AS difference FROM malnutrition GROUP BY country HAVING MAX(mean_estimate) - MIN(mean_estimate) > 0 ORDER BY difference DESC;"""
cursor.execute(query)
results=cursor.fetchall()
results

(('Equatorial Guinea', 3.003992, 15.27919375, 12.275201749999999),
 ('Saudi Arabia', 3.1890925, 15.27919375, 12.09010125),
 ('Japan', 1.3626638, 13.270979, 11.9083152),
 ('Burkina Faso', 3.5992008, 15.27919375, 11.679992949999999),
 ('Haiti', 3.6696102, 15.27919375, 11.60958355),
 ('Congo, The Democratic Republic of the', 3.988893, 15.27919375, 11.29030075),
 ('Congo', 4.4272255, 15.27919375, 10.851968249999999),
 ('South Sudan', 4.6128396, 15.27919375, 10.66635415),
 ('Somalia', 4.6888955, 15.27919375, 10.59029825),
 ('Kenya', 4.8048262, 15.27919375, 10.47436755),
 ('Zimbabwe', 2.2208489, 12.500335, 10.2794861),
 ('Mali', 5.0851859, 15.27919375, 10.194007849999998),
 ('Afghanistan', 5.122941, 15.27919375, 10.15625275),
 ('Djibouti', 4.7040578, 14.819601, 10.115543200000001),
 ('Madagascar', 5.191616, 15.27919375, 10.08757775),
 ('Maldives', 5.228566, 15.27919375, 10.05062775),
 ('Lesotho', 2.3480572, 12.089641, 9.7415838),
 ('Mozambique', 2.4245897, 12.147893, 9.7233033),
 ('Burundi',

In [25]:
#Min/Max malnutrition levels year-wise comparison
query = """SELECT year,MIN(mean_estimate) AS min_malnutrition,MAX(mean_estimate) AS max_malnutrition FROM malnutrition GROUP BY year ORDER BY year;"""
cursor.execute(query)
results=cursor.fetchall()
results

((2012, 0.17530927, 15.27919375),
 (2013, 0.17642054, 15.27919375),
 (2014, 0.17777219, 15.27919375),
 (2015, 0.17844464, 15.27919375),
 (2016, 0.17880352, 15.27919375),
 (2017, 0.17928582, 15.27919375),
 (2018, 0.17979586, 15.27919375),
 (2019, 0.18042778, 15.27919375),
 (2020, 0.18198916, 15.27919375),
 (2021, 0.18471746, 15.27919375),
 (2022, 0.18817388, 15.27919375))

In [26]:
# High CI_Width flags for monitoring(CI_width > 5)
query = """SELECT country,region,year,ci_width,mean_estimate FROM malnutrition WHERE ci_width > 5 ORDER BY ci_width DESC;"""
cursor.execute(query)
results=cursor.fetchall()
results

(('Chad', 'Africa', 2012, 13.9223397125, 11.466194),
 ('Namibia', 'Africa', 2015, 13.9223397125, 10.888676),
 ('Niger', 'Africa', 2017, 13.9223397125, 15.27919375),
 ('Central African Republic', 'Africa', 2015, 13.9223397125, 6.3030116),
 ('Comoros', 'Africa', 2015, 13.9223397125, 11.255722),
 ('Sri Lanka', 'South-East Asia', 2012, 13.9223397125, 15.27919375),
 ('Equatorial Guinea', 'Africa', 2015, 13.9223397125, 7.338772),
 ('Eritrea', 'Africa', 2017, 13.9223397125, 8.4756225),
 ('Mauritania', 'Africa', 2016, 13.9223397125, 14.300834),
 ('Trinidad and Tobago', 'Americas', 2022, 13.9223397125, 10.220496),
 ('Congo, The Democratic Republic of the',
  'Africa',
  2020,
  13.9223397125,
  9.404286),
 ('Equatorial Guinea', 'Africa', 2014, 13.9223397125, 6.2260181),
 ('Congo, The Democratic Republic of the',
  'Africa',
  2021,
  13.9223397125,
  6.8883896),
 ('South Sudan', 'Africa', 2018, 13.9223397125, 10.514835),
 ('Guinea-Bissau', 'Africa', 2013, 13.9223397125, 13.029729),
 ('Nigeria',

In [27]:
#Obesity vs malnutrition comparison by country(any 5 countries)
query = """SELECT o.country,o.year,AVG(o.mean_estimate) AS avg_obesity,AVG(m.mean_estimate) AS avg_malnutrition
FROM obesity o JOIN malnutrition m ON o.country = m.country AND o.year = m.year
WHERE o.country IN ('India', 'Nigeria', 'Brazil', 'China', 'USA')GROUP BY o.country, o.year ORDER BY o.country, o.year;"""
cursor.execute(query)
results=cursor.fetchall()
results

(('Brazil', 2012, 13.480741216666678, 2.838653299999999),
 ('Brazil', 2013, 14.112612966666653, 2.781388158333336),
 ('Brazil', 2014, 14.75046845833333, 2.735034283333335),
 ('Brazil', 2015, 15.38554568333335, 2.6989469583333334),
 ('Brazil', 2016, 16.007252433333335, 2.672241183333338),
 ('Brazil', 2017, 16.62054854166666, 2.653565950000003),
 ('Brazil', 2018, 17.23187659999999, 2.6415744916666637),
 ('Brazil', 2019, 17.84502396666667, 2.634594249999998),
 ('Brazil', 2020, 18.46843108333333, 2.6294117166666684),
 ('Brazil', 2021, 19.10013474999999, 2.6266367750000006),
 ('Brazil', 2022, 19.740666999999995, 2.626853608333335),
 ('China', 2012, 6.201950774999999, 5.08056345),
 ('China', 2013, 6.666021624999992, 4.9036836666666614),
 ('China', 2014, 7.131323083333334, 4.7372678749999935),
 ('China', 2015, 7.596237558333333, 4.5786035249999975),
 ('China', 2016, 8.063722041666672, 4.427907458333329),
 ('China', 2017, 8.540970975, 4.286700691666654),
 ('China', 2018, 9.042972741666667, 4.1

In [28]:
#Gender-based disparity in both obesity and malnutrition
query = """SELECT o.country,o.year,o.gender,AVG(o.mean_estimate) AS avg_obesity,AVG(m.mean_estimate) AS avg_malnutrition,
    (AVG(o.mean_estimate) - AVG(m.mean_estimate)) AS disparity FROM obesity o JOIN malnutrition m 
    ON o.country = m.country AND o.year = m.year AND o.gender = m.gender GROUP BY o.country, o.year, o.gender ORDER BY disparity DESC;"""
cursor.execute(query)
results=cursor.fetchall()
results

(('Niue', 2016, 'Male', 36.57509561249999, 0.7560521525, 35.819043459999996),
 ('Niue',
  2017,
  'Male',
  36.57509561249999,
  0.7691490175000002,
  35.805946594999995),
 ('Niue', 2018, 'Male', 36.57509561249999, 0.782257165, 35.792838447499996),
 ('Niue',
  2019,
  'Male',
  36.57509561249999,
  0.8000689900000002,
  35.77502662249999),
 ('Niue', 2020, 'Male', 36.57509561249999, 0.8168950975, 35.75820051499999),
 ('Niue',
  2021,
  'Male',
  36.57509561249999,
  0.8270552549999999,
  35.748040357499995),
 ('Niue', 2022, 'Male', 36.57509561249999, 0.8400994825, 35.73499612999999),
 ('Niue', 2015, 'Male', 36.343390903125, 0.7415966525000002, 35.601794250625),
 ('Nauru', 2022, 'Male', 36.57509561249999, 1.054905355, 35.52019025749999),
 ('Cook Islands',
  2022,
  'Male',
  36.57509561249999,
  1.097170965,
  35.47792464749999),
 ('Nauru', 2021, 'Male', 36.48177595937499, 1.0204185125, 35.46135744687499),
 ('Cook Islands',
  2021,
  'Male',
  36.46311220937499,
  1.0571210825,
  35.4059

In [29]:
#Region-wise avg estimates side-by-side(Africa and America)
query = """SELECT year,AVG(CASE WHEN region = 'Africa' THEN mean_estimate END) AS africa_avg,
    AVG(CASE WHEN region = 'Americas Region' THEN mean_estimate END) AS americas_avg FROM obesity GROUP BY year ORDER BY year;"""
cursor.execute(query)
results=cursor.fetchall()
results

((2012, 4.320597388806416, 17.231626583333334),
 (2013, 4.482210823962671, 17.616202000000005),
 (2014, 4.657426299240456, 18.014442250000002),
 (2015, 4.848358772092008, 18.423058166666667),
 (2016, 5.058242632734375, 18.842456750000004),
 (2017, 5.288643840807295, 19.275381749999998),
 (2018, 5.538321730464415, 19.71705025),
 (2019, 5.8072830672352405, 20.168014749999998),
 (2020, 6.0971000379297005, 20.62421766666667),
 (2021, 6.4062172268880175, 21.079996416666663),
 (2022, 6.734462902894965, 21.488845551041663))

In [30]:
#Countries with obesity up & malnutrition down
query = """WITH obesity_change AS (SELECT country,MAX(mean_estimate) - MIN(mean_estimate) AS obesity_change FROM obesity GROUP BY country),
malnutrition_change AS (SELECT country,MAX(mean_estimate) - MIN(mean_estimate) AS malnutrition_change FROM malnutrition GROUP BY country)SELECT o.country,o.obesity_change,
m.malnutrition_change FROM obesity_change o JOIN malnutrition_change m ON o.country = m.country
WHERE o.obesity_change > 0 AND m.malnutrition_change < 0 ORDER BY o.obesity_change DESC, m.malnutrition_change ASC;"""
cursor.execute(query)
results=cursor.fetchall()
results

()

In [ ]:
#Age-wise trend analysis
query = """SELECT o.year,o.age_group,AVG(o.mean_estimate) AS avg_obesity,AVG(m.mean_estimate) AS avg_malnutrition FROM obesity o
JOIN malnutrition m ON o.year = m.year AND o.age_group = m.age_group GROUP BY o.year, o.age_group ORDER BY o.year, o.age_group;"""
cursor.execute(query)
results=cursor.fetchall()
for row in results:
    print(row)